In [43]:
%%capture
! pip install pyLDAvis
! pip install pymorphy2

In [76]:
from gensim.test.utils import common_corpus
from gensim.models import LdaSeqModel
import gensim
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import warnings
import json
import pymorphy2
import nltk
import re 
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import nltk
from nltk import ngrams
import time
import dateutil.parser as dp
from sklearn.model_selection import train_test_split
nltk.download('stopwords')

russian_stopwords = stopwords.words("russian")
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
with open('output.json') as f:
    row_data = json.load(f)
len(row_data), row_data[0]

(144,
 {'link': 'https://www.tinkoff.ru/invest/social/profile/Tinkoff_Investments/324eebd5-a985-48d2-8973-815987d34d41/',
  'text': 'В прямом эфире «Деньги не спят» Василий Олейник разбирает, кто и как пострадает от нового пакета санкций ЕС, делится своей обновленной инвестиционной стратегией и отвечает на вопросы подписчиков.',
  'title': 'Деньги не спят. Удар по нефти, газу и экономике: к чему приведут новые санкции / Главные выводы для инвесторов. LIVE',
  'source': 'TINKOFF',
  'timestamp': '2022-10-07T18:49:19+03:00',
  'refers_to': ['https://youtu.be/5A85hyuGitk']})

In [68]:
class Preparator():
    def __init__(self):
        self.morph = pymorphy2.MorphAnalyzer()
        self.tokenizer = nltk.WordPunctTokenizer()
        self.regexp = re.compile('[-%-,_\.\n\\\t\r/\W]')  # оставляет только буквы и цифры
        self.pos_decoder = {"ADJF": "ADJ",
                            'ADJS': 'ADJ',
                            'COMP': 'ADJ',
                            'INFN': 'VERB',
                            'PRTF': 'VERB'}

    def norm_word(self, word: str) -> str:
        w = str(re.sub(' +', ' ', self.regexp.sub(' ', word.lower()))).strip()
        if len(w) > 1 and w[0].isalpha():
            parsed = self.morph.parse(w)[0]
            tag = str(parsed.tag.POS)
            if tag not in ['NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ']:
                tag = tag if tag not in self.pos_decoder else self.pos_decoder[tag]
                return parsed.normal_form.replace('ё', 'e')
        return ''
    
    def norm_sentence(self, sentence: str):
        tokens = list(filter(lambda x: len(x), [self.norm_word(word) for word in self.tokenizer.tokenize(sentence.lower())]))
        # bigrams = ['_'.join(w) for w in  ngrams(tokens,n=2)]
        return tokens
    
        

In [70]:
prep = Preparator()
corpus = [prep.norm_sentence(row_data_i['text']) for row_data_i in tqdm(row_data)]

  0%|          | 0/144 [00:00<?, ?it/s]

In [89]:
dictionary = gensim.corpora.Dictionary(corpus)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]
time_slice = [int(dp.parse(row_data_i['timestamp']).strftime('%m')) for row_data_i in row_data]
num_topics = 5

In [ ]:
%%time
ldaseq = LdaSeqModel(corpus=doc_term_matrix, id2word=dictionary, time_slice=time_slice, num_topics=num_topics)
with open('ldaseq2.pkl', 'wb') as f:
     pickle.dump(ldaseq, f)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldaseqmodel.py:290: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)


In [ ]:
doc_topic, topic_term, doc_lengths, term_frequency, vocab = ldaseq.dtm_vis(time=0, corpus=doc_term_matrix)
vis_dtm = pyLDAvis.prepare(topic_term_dists=topic_term, doc_topic_dists=doc_topic, doc_lengths=doc_lengths, vocab=vocab, term_frequency=term_frequency)
pyLDAvis.display(vis_dtm)